# State Task Networks

This notebook implements in Pyomo the state task network described in Section 3 of Kondili, Pantelides & Sargent (1993).

In [1]:
from pyomo.environ import *
import itertools

In [2]:
model = ConcreteModel()

# Notation

The time discretisation is defined by

In [3]:
model.H = 4
model.T = Set(initialize=RangeSet(model.H + 1), doc='Time Periods')
model.T.pprint()

T : Time Periods
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    5 : {1, 2, 3, 4, 5}


States, tasks and units are defined by

In [4]:
model.States = Set(initialize=['S1', 'S2', 'S3'], doc='States')
model.I = Set(initialize=['I1', 'I2'], doc='Tasks')
model.J = Set(initialize=['J1', 'J2'], doc='Units')

## Task definition

Task $i$ is defined by index sets, but we cannot initialize parameters or variables by indexed sets in Pyomo so we use an indexed set and tuples. We could just use the map directly, but it may be easier to specify the index set for the GUI. The user defines the index set and `mola` builds the tuples.

In [5]:
map_S_I = {'I1': ['S1', 'S2', 'S3'],
           'I2': ['S3']}
model.SI = Set(model.I, initialize=map_S_I, within=model.States)
model.S_I = Set(within=model.States * model.I, initialize=[(s, i) for i in model.I for s in model.SI[i]], 
                doc='States that feed task i')
model.S_I.pprint()

S_I : States that feed task i
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain     : Size : Members
    None :     2 : S_I_domain :    4 : {('S1', 'I1'), ('S2', 'I1'), ('S3', 'I1'), ('S3', 'I2')}


In [6]:
map_S_bar_I = {'I1': ['S1', 'S2'],
              'I2': ['S3']}
model.S_barI = Set(model.I, initialize=map_S_bar_I, within=model.States)
model.S_bar_I = Set(within=model.States * model.I, initialize=[(s, i) for i in model.I for s in model.S_barI[i]],
                   doc='States produced by task i')
model.S_bar_I.pprint()

S_bar_I : States produced by task i
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain         : Size : Members
    None :     2 : S_bar_I_domain :    3 : {('S1', 'I1'), ('S2', 'I1'), ('S3', 'I2')}


In [7]:
def rho_initialize(model, s, i):
    return 1/len(model.SI[i])
model.rho = Param(model.S_I, initialize=rho_initialize, doc='Proportion of input of task i from state s')
model.rho.pprint()

rho : Proportion of input of task i from state s
    Size=4, Index=S_I, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('S1', 'I1') : 0.3333333333333333
    ('S2', 'I1') : 0.3333333333333333
    ('S3', 'I1') : 0.3333333333333333
    ('S3', 'I2') :                1.0


In [8]:
def rho_bar_initialize(model, s, i):
    return 1/len(model.SI[i])
model.rho_bar = Param(model.S_bar_I, initialize=rho_bar_initialize, doc='Proportion of output of task i to state s')
model.rho_bar.pprint()

rho_bar : Proportion of output of task i to state s
    Size=3, Index=S_bar_I, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('S1', 'I1') : 0.3333333333333333
    ('S2', 'I1') : 0.3333333333333333
    ('S3', 'I2') :                1.0


In [9]:
model.P = Param(model.S_bar_I, initialize=1, doc='Processing time for the output of task i to state s')

In [10]:
def p_rule(model, i):
    return max(model.P[s, i] for s in model.S_barI[i])
model.p = Param(model.I, initialize=p_rule, doc='Completion time for task i')
model.p.pprint()

p : Completion time for task i
    Size=2, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
     I1 :     1
     I2 :     1


In [11]:
map_K_I = {'I1': ['J1', 'J2'],
           'I2': ['J2']}
model.KI = Set(model.I, initialize=map_K_I, within=model.J)
model.K_I = Set(within=model.J * model.I, initialize=[(j, i) for i in model.I for j in model.KI[i]], 
                doc='Units capable of performing task i')

## State definition

The set $T_s$ is the set of tasks that receive material from state $s$. It is the inversion of the set $S_i$.

In [12]:
map_T_S = {}
for s, i in model.S_I:
    map_T_S.setdefault(s, []).append(i)

map_T_S

{'S1': ['I1'], 'S2': ['I1'], 'S3': ['I1', 'I2']}

In [13]:
model.TS = Set(model.States, initialize=map_T_S, within=model.I)
model.T_S = Set(within=model.I * model.States, initialize=[(i, s) for s in model.States for i in model.TS[s]], 
                doc='Tasks receiving material from state s')

In [14]:
map_T_bar_S = {}
for s, i in model.S_bar_I:
    map_T_bar_S.setdefault(s, []).append(i)

map_T_bar_S

{'S1': ['I1'], 'S2': ['I1'], 'S3': ['I2']}

In [15]:
model.T_barS = Set(model.States, initialize=map_T_bar_S, within=model.I)
model.T_bar_S = Set(within=model.I * model.States, initialize=[(i, s) for s in model.States for i in model.T_barS[s]],
                   doc='Tasks producing material in state s')
model.C = Param(model.States, initialize=1, doc='Maximum storage capacity dedicated to state s')

## Unit definition

The set $I_j$ is the set of tasks that can be performed by unit $j$. It is the inversion of the set $K_i$.

In [16]:
map_I_J = {}
for j, i in model.K_I:
    map_I_J.setdefault(j, []).append(i)

In [17]:
model.IJ = Set(model.J, initialize=map_I_J, within=model.I)
model.I_J = Set(within=model.I * model.J, initialize=[(i, j) for j in model.J for i in model.IJ[j]],
                doc='Tasks that can be performed by unit j')
model.I_J.pprint()

I_J : Tasks that can be performed by unit j
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain     : Size : Members
    None :     2 : I_J_domain :    3 : {('I1', 'J1'), ('I1', 'J2'), ('I2', 'J2')}


In [18]:
model.IJ['J2'].pprint()

{Member of IJ} : Size=2, Index=J, Ordered=Insertion
    Key : Dimen : Domain : Size : Members
     J2 :     1 :      I :    2 : {'I1', 'I2'}


In [19]:
model.V_max = Param(model.I_J, initialize=1, doc='Maximum capacity of unit j when used for performing task i')
model.V_min = Param(model.I_J, initialize=0, doc='Minimum capacity of unit j when used for performing task i')

# Variables

In [20]:
model.W = Var(model.I_J, model.T, doc='Start task k using unit j at time t', within=Binary)
model.W.pprint()

W : Start task k using unit j at time t
    Size=15, Index=W_index
    Key             : Lower : Value : Upper : Fixed : Stale : Domain
    ('I1', 'J1', 1) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 2) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 3) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 4) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J1', 5) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 1) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 2) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 3) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 4) :     0 :  None :     1 : False :  True : Binary
    ('I1', 'J2', 5) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J2', 1) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J2', 2) :     0 :  None :     1 : False :  True : Binary
    ('I2', 'J2', 3) :     0 :  None 

In [21]:
model.B = Var(model.I_J, model.T, doc='Amount of material which starts undergoing task k in unit j at beginning of t')
model.S = Var(model.States, model.T, doc='Amount of material stored in state s at beginning of t')

# Constraints

## Allocation Constraints

In [22]:
model.M = Param(initialize=1000, doc='Allocation constraint parameter')
def allocation_constraints_rule(model, i, j, t):
    lhs = sum(model.W[idash, j, tdash] for tdash in model.T for idash in model.IJ[j] 
              if tdash >= t and tdash <= t + model.p[i] - 1) - 1
    rhs = model.M * (1 - model.W[i, j, t])
    return lhs <= rhs
model.allocation_constraints = Constraint(model.I_J, model.T, rule=allocation_constraints_rule)

Note that we use the indexed set `model.IJ` to sum over tasks for fixed unit $j$.

## Capacity Limitations

In [23]:
def capacity_limitations_rule(model, j, i, t):
    return inequality(model.W[i, j, t] * model.V_min[i, j], model.B[i, j, t], model.B[i, j, t] <= model.W[i, j, t] * model.V_max[i, j]) 
model.capacity_limitations = Constraint(model.K_I, model.T, rule=capacity_limitations_rule)

## Material Balances

In [24]:
model.T_barS.pprint()

T_barS : Size=3, Index=States, Ordered=Insertion
    Key : Dimen : Domain : Size : Members
     S1 :     1 :      I :    1 : {'I1',}
     S2 :     1 :      I :    1 : {'I1',}
     S3 :     1 :      I :    1 : {'I2',}


In [25]:
def material_balances_rule(model, s, t):
    if t > model.T.first():
        rhs = model.S[s, t-1]
        rhs += sum(model.rho_bar[s, i] * model.B[i, j, t-model.P[s, i]] 
                   for i in model.T_barS[s] for j in model.KI[i])
        rhs -= sum(model.rho[s, i] * model.B[i, j, t] for i in model.TS[s] for j in model.KI[i])    
        return model.S[s, t] == rhs
    else:
        return Constraint.Feasible
model.material_balances = Constraint(model.States, model.T, rule=material_balances_rule)

## Product deliveries and raw material receipts during the horizon

This is a modification of the previous constraint that we ignore here.

## Temporary Unavailability of Equipment

We don't need this constraint for the example.

## Limited Availability of Utilities and Man-Power

We don't need this constraint for the example.

## Cleaning Equipment Items

We don't need this constraint for the example.

# Objective Function

The objective for the paper is the maximisation of profit.

In [29]:
def cost_rule(model):
    cost = 1
    return cost
model.Cost = Param(model.States, model.T, initialize=cost_rule,
                   doc='Unit price of material in state s at time t')

In [30]:
def maximise_profit_rule(model):
    value_products = sum(model.Cost[s, model.H+1] * model.S[s, model.H+1] +
              sum(model.Cost[s, t] * model.D[s, t] for t in range(1, model.H+1)) for s in model.States)
    return value_products
model.obj = Objective(rule=maximise_profit_rule, doc='Maximisation of Profit')

    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for objective obj:
    AttributeError: 'ConcreteModel' object has no attribute 'D'
ERROR: Constructing component 'obj' from data=None failed: AttributeError:
    'ConcreteModel' object has no attribute 'D'


AttributeError: 'ConcreteModel' object has no attribute 'D'